In [22]:
from secreto import API_KEY
import os
from googleapiclient.discovery import build
import pandas as pd
import matplotlib.pyplot as plt

def primera_pagina(VideoId):

    youtube = build(
        "youtube", "v3", developerKey = API_KEY)

    request = youtube.commentThreads().list(
        part="id, snippet, replies",
        videoId=VideoId,
        maxResults = 100
    )
    respuesta = request.execute()

    return respuesta

def siguiente_pagina(VideoId,nextPageToken):
    
    youtube = build(
        "youtube", "v3", developerKey = API_KEY)

    request = youtube.commentThreads().list(
        part = "id, snippet, replies",
        videoId = VideoId,
        pageToken = nextPageToken,
        maxResults = 100
    )
    response = request.execute()

    return response

def transformacion(respuesta):
    
    df = pd.DataFrame(columns=["autor","texto"])
    
    for n in range(0,len(respuesta["items"])):
        autor = respuesta["items"][n]["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"]
        texto = respuesta["items"][n]["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        df.loc[n,"autor"] = autor
        df.loc[n,"texto"] = texto

    return df


def crear_dataframe(VideoId):

    respuesta = primera_pagina(VideoId)
    
    df = transformacion(respuesta)
    
    while 'nextPageToken' in respuesta.keys():
        respuesta = siguiente_pagina(VideoId,respuesta['nextPageToken'])
        df2 = transformacion(respuesta)
        df = pd.concat([df,df2], axis=0)

    return df

def contar(VideoId):

    df = crear_dataframe(VideoId)

    diccionario = {
        "4556":df["texto"].str.contains("4556").sum(),
        "4557":df["texto"].str.contains("4557").sum(),
        "4558":df["texto"].str.contains("4558").sum(),
        "4559":df["texto"].str.contains("4559").sum()
        }
    
    if diccionario["4558"] == 0:
        diccionario.pop("4558")

    if diccionario["4559"] == 0:
        diccionario.pop("4559")

    porcentajes = porcentaje(diccionario)

    return diccionario, porcentajes

def porcentaje(diccionario):

    total = diccionario["4556"] + diccionario["4557"]
    seis = str(round(diccionario["4556"] * 100 / total,2)) + " %"
    siete = str(round(diccionario["4557"] * 100 / total,2)) + " %"

    porcentajes = {"4556":seis, "4557":siete}

    return porcentajes

In [23]:
VideoId = "pMYLWBZlEKA" # 16-10-22 ({'4556': 88, '4557': 43}, {'4556': '67.18 %', '4557': '32.82 %'})
VideoId = "ZoIL42XmNlY" # 23-10-22 ({'4556': 68, '4557': 50, '4558': 8, '4559': 4}, {'4556': '57.63 %', '4557': '42.37 %'})
VideoId = "NcIsIcPG7Lg" # 30-10-22 ({'4556': 53, '4557': 31, '4558': 13}, {'4556': '63.1 %', '4557': '36.9 %'})
VideoId = "unCpPLTniI8" # 06-11-22 ({'4556': 25, '4557': 46, '4558': 6}, {'4556': '35.21 %', '4557': '64.79 %'})
VideoId = "ZdHiVyqD7Qk" # 13-11-22 ({'4556': 22, '4557': 16, '4558': 3}, {'4556': '57.89 %', '4557': '42.11 %'})


print(contar(VideoId))

({'4556': 22, '4557': 16, '4558': 3}, {'4556': '57.89 %', '4557': '42.11 %'})


- evitar que se tome mas de un tipo de voto por usuario
- interfaz gráfica
- exportar resultados a excel histórico